In [1]:
import os
os.add_dll_directory(r"D:\programming languages\anaconda\Lib\site-packages\clidriver\bin") 

<AddedDllDirectory('D:\\programming languages\\anaconda\\Lib\\site-packages\\clidriver\\bin')>

In [2]:
from flask import Flask, render_template, request, session
import cv2
import pickle
import cvzone
import numpy as np
import re
import ibm_db

In [3]:
app = Flask(__name__)

In [4]:
ibm_db.connect("DATABASE=vali1;HOSTNAME=localhost;PORT=25000;PROTOCOL=TCPIP;UID=Mansoor;PWD=mansoor@31;", "", "")
print("connected")

connected


In [5]:
@app.route('/')
def project():
       return render_template('Login.html')
    
@app.route('/login')
def login():
       return render_template('Login.html')
    
@app.route('/signup')
def signup():
       return render_template('Signup.html')
    
@app.route('/index') 
def index():
    return render_template('Index.html')

@app.route('/model')
def model():
       return render_template('Model.html')

In [6]:

def signup():
    msg = ' '
    if request.method == 'POST':
        username= request.form["username"]
        email = request.form["email"]
        password = request.form["password"]
        confirm_password = request.form["confirm_password"]
        sql = "SELECT * FROM REGISTER WHERE name = ?"
        stmt = ibm_db.prepare(conn, sql) 
        ibm_db.bind_param(stmt, 1, username)
        ibm_db.execute(stmt)
        account = ibm_db.fetch_assoc(stmt) 
        print (account)
        if account:
            return render_template('Login.html', error=True) 
        elif not re.match(r'[^@]+@[^@]+\.[^@]+', email): 
            msg = "Invalid Email Address!"
        else:
            insert_sql = "INSERT INTO REGISTER VALUES (?,?,?)"
            prep_stmt = ibm_db.prepare(conn, insert_sql)
            # this username & password should be same as db-2 details & order also
            ibm_db.bind_param(prep_stmt, 1, username)
            ibm_db.bind_param(prep_stmt, 2, email) 
            ibm_db.bind_param(prep_stmt, 3, password) 
            ibm_db.bind_param(prep_stmt, 4, confirm_password) 
            ibm_db.execute(prep_stmt)
            msg = "You have successfully registered !" 
            return render_template('Login.html', msg=msg)

In [7]:
@app.route("/log", methods=['POST', 'GET'])
def login1():
    if request.method == "POST":
        username = request.form["username"]
        password = request.form["password"]
        sql = "SELECT * FROM REGISTER WHERE EMAIL=? AND PASSWORD=?" # from db2 sql table 
        stmt = ibm_db.prepare(conn, sql)
        # this username & password should be same as db-2 details & order also
        ibm_db.bind_param(stmt, 1, username)
        ibm_db.bind_param(stmt, 2, password)
        ibm_db.execute(stmt)
        account = ibm_db.fetch_assoc(stmt)
        print (account)
        if account:
            session['Loggedin'] = True
            session['id'] = account['USERNAME']
            session['username'] = account['USERNAME'] 
            return render_template('Index.html')
        else:
            msg = "Incorrect username/password"
            return render_template('Login.html', msg=msg)
    else:
            return render_template('Login.html')

In [8]:

@app.route('/predict_live') 
def liv_pred():
    # Video feed
    cap = cv2.VideoCapture("D:\programs\Python programs\smart internz\AI car Parking\flask\carParkingInput.mp4") 
    with open('parkingSlotPosition', 'rb') as f: 
         posList = pickle.load(f)
    width, height = 107, 48
    def checkParkingSpace (imgPro):
        spaceCounter = 0
        for pos in posList:
            x, y = pos
            imgCrop = imgPro[y:y + height, x:x + width]
            # cv2.imshow(str(x * y), imgCrop)
            count = cv2.count_NonZero (imgCrop)
            if count < 900:
                color = (0, 255, 0)
                thickness = 5
                spaceCounter += 1
            else:
                color = (0, 255, 0)
                thickness = 2
            cv2.rectangle(img, pos, (pos[0] + width, pos[1] + height), color, thickness)
            cvzone.putTextRect(img, f'Free: {spaceCounter}/{len(posList)}' (100, 50), scale=3, thickness=5, offset=20, colorR=(0, 200, 0))
    while True:
         if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            success, img = cap.read()
            imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            imgBlur = cv2.GaussianBlur (imgGray, (3, 3), 1)
            imgThreshold = cv2.adaptiveThreshold(imgBlur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 25, 16)
            imgMedian = cv2.medianBlur (imgThreshold, 5)
            kernel = np.ones((3, 3), np.uint8)
            imgDilate = cv2.dilate(imgMedian, kernel, iterations=1)
            checkParkingSpace (imgDilate)  
            cv2.imshow("Image", img)
            # cv2.imshow("ImageBlur", imgBlur)
            # cv2.imshow("ImageThres", imgMedian) 
            if cv2.waitKey(1) & 0xFF== ord('q'):
                break

<>:23: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
<>:23: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
C:\Users\Mansoorvali\AppData\Local\Temp\ipykernel_16024\1799227076.py:23: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
  cvzone.putTextRect(img, f'Free: {spaceCounter}/{len(posList)}' (100, 50), scale=3, thickness=5, offset=20, colorR=(0, 200, 0))


In [9]:
if __name__=="__main__":
      app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

D:\programming languages\anaconda\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
